**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri May 20 06:56:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 95aryan.zip		        mask_rcnn_coco.h5
 alpha.ipynb		        multiple_camera_streaming01.py
'Colab Notebooks'	       'multi_streaming (1).py'
 darknet		        multi_streaming.py
 detectron2_repo	       'My Drive'
 face_detect_multi_streams.py   NewBPCL
 fire_1.mp4		        s3upload_fire.py
 fire_2.mp4		       'test_video (1).py'
 hol_1.mp4		        test_video.py
 HOL.avi		        Untitled0.ipynb
 iiq.colab.1@intersectiq.com    yolov3
 inspect_mrcnn_results.ipynb    Yolov3-tiny-training.ipynb


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15420, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 15420 (delta 1), reused 6 (delta 1), pack-reused 15413
Receiving objects: 100% (15420/15420), 14.05 MiB | 17.96 MiB/s, done.
Resolving deltas: 100% (10358/10358), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "load" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-11-09 13:55:56--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.9MB/s    in 8.3s    

2021-11-09 13:56:05 (18.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/TestImage0_0.jpg  
  inflating: data/obj/TestImage0_0.txt  
  inflating: data/obj/TestImage0_1.jpg  
  inflating: data/obj/TestImage0_1.txt  
  inflating: data/obj/TestImage0_10.jpg  
  inflating: data/obj/TestImage0_10.txt  
  inflating: data/obj/TestImage0_11.jpg  
  inflating: data/obj/TestImage0_11.txt  
  inflating: data/obj/TestImage0_12.jpg  
  inflating: data/obj/TestImage0_12.txt  
  inflating: data/obj/TestImage0_13.jpg  
  inflating: data/obj/TestImage0_13.txt  
  inflating: data/obj/TestImage0_14.jpg  
  inflating: data/obj/TestImage0_14.txt  
  inflating: data/obj/TestImage0_15.jpg  
  inflating: data/obj/TestImage0_15.txt  
  inflating: data/obj/TestImage0_16.jpg  
  inflating: data/obj/TestImage0_16.txt  
  inflating: data/obj/TestImage0_17.jpg  
  inflating: data/obj/TestImage0_17.txt  
  inflating: data/obj/TestImage0_18.jpg  
  inflating: data/obj/TestImage0_18.txt  
  inf

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
257 data/obj/TestImage2106_16.txt
0 0.543527 0.175987 0.037202 0.111184
15 0.554721 0.461417 0.345494 0.308661

['15', '0.554721', '0.461417', '0.345494', '0.308661']
258 data/obj/TestImage1029_19.txt
0 0.554721 0.461417 0.345494 0.308661
15 0.480655 0.229934 0.050595 0.104605

['15', '0.480655', '0.229934', '0.050595', '0.104605']
259 data/obj/TestImage3459_28.txt
0 0.480655 0.229934 0.050595 0.104605
15 0.221168 0.192763 0.096354 0.088158

['15', '0.221168', '0.192763', '0.096354', '0.088158']
260 data/obj/TestImage3118_23.txt
0 0.221168 0.192763 0.096354 0.088158
15 0.810640 0.269408 0.090774 0.146711

['15', '0.810640', '0.269408', '0.090774', '0.146711']
261 data/obj/TestImage2854_10.txt
0 0.810640 0.269408 0.090774 0.146711
15 0.395647 0.286513 0.039062 0.111184

['15', '0.395647', '0.286513', '0.039062', '0.111184']
262 data/obj/TestImage3855_14.txt
0 0.395647 0.286513 0.039062 0.111184
15 0.511719 0.169079 0.038318 0.118421

['

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/TestImage1059_10.jpg', 'data/obj/TestImage304_3.jpg', 'data/obj/TestImage1_12.jpg', 'data/obj/TestImage2920_27.jpg', 'data/obj/TestImage3459_12.jpg', 'data/obj/TestImage3162_24.jpg', 'data/obj/TestImage304_0.jpg', 'data/obj/TestImage2634_4.jpg', 'data/obj/TestImage1062_21.jpg', 'data/obj/TestImage1059_22.jpg', 'data/obj/TestImage2854_9.jpg', 'data/obj/TestImage2018_22.jpg', 'data/obj/TestImage1062_14.jpg', 'data/obj/TestImage1732_24.jpg', 'data/obj/TestImage1092_20.jpg', 'data/obj/TestImage2227_23.jpg', 'data/obj/TestImage3338_6.jpg', 'data/obj/TestImage3855_12.jpg', 'data/obj/TestImage2711_14.jpg', 'data/obj/TestImage1512_1.jpg', 'data/obj/TestImage216_22.jpg', 'data/obj/TestImage1124_6.jpg', 'data/obj/TestImage1919_22.jpg', 'data/obj/TestImage2315_15.jpg', 'data/obj/TestImage1633_12.jpg', 'data/obj/TestImage1026_20.jpg', 'data/obj/TestImage3679_7.jpg', 'data/obj/TestImage3903_1.jpg', 'data/obj/TestImage216_11.jpg', 'data/obj/TestImage2634_22.jpg', 'data/obj/TestImage3679_1

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.755513), count: 3, class_loss = 0.158532, iou_loss = 0.140208, total_loss = 0.298740 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.095082, iou_loss = 0.000000, total_loss = 0.095082 
 total_bbox = 71115, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.676427), count: 4, class_loss = 0.157121, iou_loss = 0.339178, total_loss = 0.496299 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.010174, iou_loss = 0.000000, total_loss = 0.010174 
 total_bbox = 71119, rewritten_bbox = 0.000000 % 

 1242: